# Artificial Intelligence II: HW1 Tutorial

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import pandas as pd

## Loading and Exploring the Dataset

In [73]:
train_df = pd.read_csv("VaccineSentimentDataset/vaccine_train_set.csv", index_col=0)
train_df

,tweet,label
0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,I don't know about you but My family and I wil...,1
2,@MSignorile Immunizations should be mandatory....,2
3,President Obama spoke in favor of vaccination ...,0
4,"""@myfoxla: Arizona monitoring hundreds for mea...",0
...,...,...
15971,@Salon if u believe the anti-vax nutcases caus...,1
15972,How do you feel about parents who don't #vacci...,0
15973,70 Preschoolers Tested for Measles in Simi Val...,0
15974,Finance Minister: Budget offers room to procur...,0


In [74]:
train_df.describe()

,label
count,15976.000000
mean,0.936592
std,0.930740
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,2.000000


In [75]:
test_df = pd.read_csv("VaccineSentimentDataset/vaccine_validation_set.csv", index_col=0)
test_df

,tweet,label
0,@user They had a massive surge in with covid d...,1
1,Required vaccines for school: Parents and guar...,0
2,“@KCStar: Two more Johnson County children hav...,0
3,NV can do better. Which states are the best (a...,2
4,Nothing like killing ourselves w/ our own fear...,2
...,...,...
2277,RT @abc7: Number of measles cases reported in ...,0
2278,"Evidence points to the idea that ""measles affe...",0
2279,"Where's @SavedYouAClick ""@voxdotcom: Why you s...",2
2280,Some of my favorite people have autism. If tha...,2


In [76]:
test_df.describe()

,label
count,2282.000000
mean,0.936897
std,0.930960
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,2.000000


## Visualize Data

In [ ]:
import seaborn as sn
sn.countplot(x='label',data=train_df)

In [ ]:
sn.countplot(x='label',data=test_df)

## Data Pre-processing

In [77]:
#Check for null values
train_df.isnull().sum()

tweet    0
label    0
dtype: int64

In [78]:
test_df.isnull().sum()

tweet    0
label    0
dtype: int64

#### Separating features from targets

In [79]:
X_train = train_df.drop('label', axis=1)
Y_train = train_df['label']
X_train

,tweet
0,Sip N Shop Come thru right now #Marjais #Popul...
1,I don't know about you but My family and I wil...
2,@MSignorile Immunizations should be mandatory....
3,President Obama spoke in favor of vaccination ...
4,"""@myfoxla: Arizona monitoring hundreds for mea..."
...,...
15971,@Salon if u believe the anti-vax nutcases caus...
15972,How do you feel about parents who don't #vacci...
15973,70 Preschoolers Tested for Measles in Simi Val...
15974,Finance Minister: Budget offers room to procur...


In [80]:
Y_train

0        0
1        1
2        2
3        0
4        0
        ..
15971    1
15972    0
15973    0
15974    0
15975    2
Name: label, Length: 15976, dtype: int64

In [81]:
X_test = test_df.drop('label', axis=1)
Y_test = test_df['label']
X_test

,tweet
0,@user They had a massive surge in with covid d...
1,Required vaccines for school: Parents and guar...
2,“@KCStar: Two more Johnson County children hav...
3,NV can do better. Which states are the best (a...
4,Nothing like killing ourselves w/ our own fear...
...,...
2277,RT @abc7: Number of measles cases reported in ...
2278,"Evidence points to the idea that ""measles affe..."
2279,"Where's @SavedYouAClick ""@voxdotcom: Why you s..."
2280,Some of my favorite people have autism. If tha...


In [82]:
Y_test

0       1
1       0
2       0
3       2
4       2
       ..
2277    0
2278    0
2279    2
2280    2
2281    0
Name: label, Length: 2282, dtype: int64

#### Text pre-processing

In [95]:
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize
from textblob import TextBlob
from emot import emo_unicode
import re


def preprocess(dataset):
    # remove uppercase letters
    dataset['tweet'] = dataset['tweet'].apply(lambda s: s.lower() if type(s) == str else s)

    # convert emojis and emoticons to text
    dataset['tweet'] = dataset['tweet'].apply(emojisToWords)
    #dataset['tweet'] = dataset['tweet'].apply(emoticonsToWords)

    # remove URLs
    dataset['tweet'] = dataset['tweet'].replace(to_replace=r'^https?:\/\/.*[\r\n]*', value='', regex=True)

    # remove numbers
    dataset['tweet'] = dataset['tweet'].replace(to_replace=r'\d+', value='', regex=True)

    # remove \n, \u0111
    dataset['tweet'] = dataset['tweet'].replace(to_replace='\n', value='', regex=True)
    dataset['tweet'] = dataset['tweet'].replace(to_replace='\u0111', value='', regex=True)

    # remove punctuation
    dataset['tweet'] = dataset['tweet'].str.replace('[^\w\s]', '')  # replace(r'^https?:\/\/.*[\r?\n\t@]*', '')

    # remove stopwords
    dataset['tweet'] = dataset['tweet'].apply(removeStopwords)

    # removes most common words
    cnt = findMostCommonWords(dataset)
    dataset['tweet'] = dataset["tweet"].apply(removeCommonWords)

    # Rare word removal

    # spelling correction
    dataset['tweet'].apply(lambda x: str(TextBlob(x).correct()))

    for entry in dataset:
        # remove whitespaces
        entry = entry.strip()
        # remove stopwords and tokenize
        tokens = word_tokenize(entry)
        entry = [i for i in tokens if not i in stopwords]

    return dataset

    # remove symbols
    # df['Comment'] = df['Comment'].replace(to_replace =r'([^\s\w]|_)+', value = '', regex = True)


def emojisToWords(text):
    for emot in emo_unicode.EMOJI_UNICODE:
        text = text.replace(emot, "_".join(emo_unicode.EMOJI_UNICODE[emot].replace(",", "").replace(":", "").split()))
        return text


def emoticonsToWords(text):
    for emot in emo_unicode.EMOTICONS_EMO:
        text = re.sub(u'(' + emot + ')', "_".join(emo_unicode.EMOTICONS_EMO[emot].replace(",", "").split()), text)
        return text


def removeStopwords(text):
    STOPWORDS = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if
                     word not in STOPWORDS])  # Applying the stopwords to 'text_punct' and store into 'text_stop'


def findMostCommonWords(dataset):
    cnt = Counter()
    for text in dataset["text_stop"].values:
        for word in text.split():
            cnt[word] += 1

    return cnt


def removeCommonWords(text, toRemove, cnt):
    return " ".join([word for word in str(text).split() if word not
                     in cnt.most_common(toRemove)])


In [ ]:
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize
from textblob import TextBlob
from emot import emo_unicode
import re

from Preprocess import *

def preprocess(dataset):

    # remove uppercase letters
    dataset['tweet'] = dataset['tweet'].apply(lambda s: s.lower() if type(s) == str else s)

    # convert emojis and emoticons to text
    dataset['tweet'] = dataset['tweet'].apply(emojisToWords)
    #dataset['tweet'] = dataset['tweet'].apply(emoticonsToWords)

    # remove URLs
    dataset['tweet'] = dataset['tweet'].replace(to_replace=r'^https?:\/\/.*[\r\n]*', value='', regex=True)

    # remove numbers
    dataset['tweet'] = dataset['tweet'].replace(to_replace=r'\d+', value='', regex=True)

    # remove \n, \u0111
    dataset['tweet'] = dataset['tweet'].replace(to_replace='\n', value='', regex=True)
    dataset['tweet'] = dataset['tweet'].replace(to_replace='\u0111', value='', regex=True)

    # remove punctuation
    dataset['tweet'] = dataset['tweet'].str.replace('[^\w\s]', '')  # replace(r'^https?:\/\/.*[\r?\n\t@]*', '')

    # remove stopwords
    dataset['tweet'] = dataset['tweet'].apply(removeStopwords)

    # removes most common words
    #cnt = findMostCommonWords(dataset)
    #dataset['tweet'] = dataset["tweet"].apply(removeCommonWords)

    # Rare word removal

    # spelling correction
    dataset['tweet'].apply(lambda x: str(TextBlob(x).correct()))

    return dataset

X_train = preprocess(X_train)
X_train

/tmp/ipykernel_38215/455184214.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['tweet'] = dataset['tweet'].str.replace('[^\w\s]', '')  # replace(r'^https?:\/\/.*[\r?\n\t@]*', '')


In [ ]:
X_test = preprocess(X_test)
X_test

#### Vectorize words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#with unigrams without any optimization for now

vectorizer = CountVectorizer()
vectorizer.fit(X_train['tweet'])

train_vec = vectorizer.transform(X_train['tweet'])
print(vectorizer.get_feature_names())

In [ ]:
#with unigrams without any optimization for now

vectorizer = CountVectorizer()
vectorizer.fit(X_test['tweet'])

test_vec = vectorizer.transform(X_test['tweet'])
print(vectorizer.get_feature_names())

## Train model

In [ ]:
#We will experiment with Ridge Regression
clf = LogisticRegression()
clf.fit(train_vec, Y_train)

## Evaluate model's performance

In [ ]:
# Now that we have trained the classifer, we can make predictions on the unseen data
Y_test_pred = clf.predict(test_vec)
print(Y_test_pred)
# Let's also make predictions on the train set for reference
Y_train_pred = clf.predict(train_vec)

In [ ]:
test_mse = mean_squared_error(Y_test, Y_test_pred)
print(f"Our classifier achieves a MSE of {test_mse:.2f} on the test set")
train_mse = mean_squared_error(Y_train, Y_train_pred)
print(f"Our classifier achieves a MSE of {train_mse:.2f} on the train set")

## Notes

* Scaling features
* Experimenting with different models 
* Using different hyperparameters for each model
* Testing which of the features are really helpful
* Creating additional synthetic features
* And many more...
